In [1]:
import pandas as pd
def general(stockiid = '2330'):
    stockiid = int(stockiid)
    Result_Str = ''
    CSV = './all_stock_price/public/{}.TW.csv'.format(stockiid)

    daily_trade =pd.read_csv('{}'.format(CSV))
    daily_trade = daily_trade.to_dict()

    Dic_Open = {}
    Dic_High = {}
    Dic_Low = {}
    Dic_Close = {}
    Dic_Volume = {}


    for count in range(len(daily_trade['Date'])):
        Dic_Open[daily_trade['Date'][count]] = daily_trade['Open'][count]
        Dic_High[daily_trade['Date'][count]] = daily_trade['High'][count]
        Dic_Low[daily_trade['Date'][count]] = daily_trade['Low'][count]
        Dic_Close[daily_trade['Date'][count]] = daily_trade['Close'][count]
        Dic_Volume[daily_trade['Date'][count]] = daily_trade['Volume'][count]

    L = [l for l in Dic_Close.values()]
    D = [d for d in daily_trade['Date'].values()]


    def Moving_Average(interval = 5):
        Dic_MA = {}
        interval  =int(interval)
        for i in range(len(L)):
            if i-interval <0:
                pass
            else:
                Dic_MA[D[i]]="{:}".format(sum(L[i-interval:i])/interval)
        return Dic_MA


    def Bias(days = 3,date = '2020-02-19'):
        try:
            x = Dic_Close[date]
            y = float( Moving_Average( interval = days )[date] )
            Result = (x-y)/y
        except KeyError as f:
            Result = 100.0
        return Result


    def EMA(day = 12):

        Result_List = []
        Result_Dic = {}
        day  =int(day)
        for i in range(len(L)):
            denominator_EMA = 0
            fraction_EMA = 0
            if i-day < 0:
                Result_List.append('NaN')
            else:
                interval_price = L[i - day:i]
                alpha = ((2/(day+1)))
                for c in range(day):
                    denominator_EMA += (1-alpha)**c
                    fraction_EMA += ((1-alpha)**c)*float(interval_price[c])
                Result_List.append (fraction_EMA / denominator_EMA )
                Result_Dic = dict(zip(D,Result_List))
        return Result_Dic



    def DIF(day1 = 12,day2 = 26):
        day1 = int(day1)
        day2 = int(day2)
        Result_List = []
        EMA_1 = [e for e in EMA(day = day1).values()]
        EMA_2 = [e for e in EMA(day = day2).values()]
        for c in range(len(EMA_1)):
            try:
                Result_List.append(EMA_1[c] - EMA_2[c])
            except TypeError as e:
                Result_List.append('NaN')
            Result_Dic = dict(zip(D,Result_List))
        return Result_Dic

    def MACD(day = 9):
        mL = [l for l in DIF(day1 = 12,day2 = 26).values()]
        mResult_List = []
        mResult_Dic = {}
        day  =int(day)
        for i in range(len(L)):
            denominator_EMA = 0
            fraction_EMA = 0
            if i-day < 0:
                mResult_List.append('NaN')

            else:
                interval_price = mL[i - day:i]
                alpha = ((2/(day+1)))
                for c in range(day):
                    denominator_EMA += (1-alpha)**c
                    fraction_EMA += ((1-alpha)**c)*float(interval_price[c])
                mResult_List.append (fraction_EMA / denominator_EMA )
                mResult_Dic = dict(zip(D,mResult_List))
        return mResult_Dic


    def RSV(n = 9):
        Close_List = [l for l in Dic_Close.values()]
        High_List = [h for h in Dic_High.values()]
        Low_list = [low for low in Dic_Low.values()]
        D = [d for d in Dic_Close.keys()]
        day = int(n)
        Result_List = []
        for i in range(len(Close_List)):
            denominator_RSV = 0
            fraction_RSV = 0
            min_price = 0.0
            Max_price = 0.0
            Result = 0.0
            if i-day < 0:
                Result_List.append('NaN')
            else:
                interval_close_price = Close_List[i - day:i]
                interval_high_price = High_List[i - day:i]
                interval_low_price = Low_list[i - day:i]
                min_price = min(interval_low_price)
                Max_price = max(interval_high_price)

                denominator_RSV = Max_price - min_price
                fraction_RSV = Close_List[i] - min_price
                Result = fraction_RSV / denominator_RSV
                Result_List.append(Result)
        Result_dict = dict(zip(D,Result_List))
        return Result_dict


    def K_values(dic = RSV()):
        Days = [d for d in dic.keys()][9:]
        Null_list = ['NaN' for i in range(9)]
        K_value = 0.0
        K_list = []
        for day in Days:
            K_value = (K_value)*(2/3)+ (dic[day])*(1/3)
            K_list.append(K_value)
        K_list = ['NaN' for i in range(9)] + K_list
        Result_Dic = dict(zip([d for d in dic.keys()],K_list))
        return Result_Dic



    def D_values():
        Result_Dic = K_values(dic = K_values())
        return Result_Dic


    def Mean_Volume(interval = 15):
        L = [l for l in Dic_Volume.values()]
        D = [d for d in Dic_Volume.keys()]
        interval = int(interval)
        Dic_MV = {}
        for i in range(len(L)):
            if i - interval <0:
                pass
            else:
                Dic_MV[D[i]]="{}".format(sum(L[i-interval:i])/interval)
        return Dic_MV


    def All_Average_good():
        Result = []
        All_Moving_Average = {}
        for date in Moving_Average(60):
            All_Moving_Average[date] = [Moving_Average(5)[date],Moving_Average(10)[date],Moving_Average(20)[date],Moving_Average(60)[date]]

        for date in All_Moving_Average.keys():
            M5 = All_Moving_Average[date][0]
            M10 = All_Moving_Average[date][1]
            M20 = All_Moving_Average[date][2]
            M60 = All_Moving_Average[date][3]
            if M5 >= max(All_Moving_Average[date]) and M60 <= min(All_Moving_Average[date]) and M10 >= M20:
                Result.append(date)
        return Result


    def price_good(price_dict = Dic_Close , day1 = 5 , day2 = 10 , day3 = 20 ,day4 = 60):
        day1,day2,day3,day4 = int(day1),int(day2),int(day3),int(day4)
        Result = []
        All_Moving_Average = {}
        for date in Moving_Average(day4):
            if price_dict[date] > max(float(Moving_Average(day1)[date]),float(Moving_Average(day2)[date]),float(Moving_Average(day3)[date]),float(Moving_Average(day4)[date])):
                Result.append(date)
        return Result


    def MACD_good(n = 9):
        n = int(n)
        func = MACD(day = n).copy()
        Result_list = []
        del_list = []
        for t in func.keys():
            if type(func[t]) == str:
                del_list.append(t)
        for d in del_list:
            del func[d]
        times = [t for t in func.keys()]
        prices = [p for p in func.keys()]

        for i in range(len(times)):
            try:
                if func[times[i]] < 0 and func[times[i+1]] > 0:
                    Result_list.append(times[i])
            except IndexError as e:
                pass
        return Result_list


    def Volume_good(day = 15):
        day = int(day)
        dates = [d for d in Mean_Volume(interval = day).keys()]
        Result_List = []
        for date in dates:
            if float(Dic_Volume[date]) > float(Mean_Volume(interval = day)[date]):
                Result_List.append(date)
        return Result_List


    def Gold_Cross(difference = 0.1):
        Result_List = []
        difference = float(difference)
        for d in K_values():
            if float(K_values()[d]) - float(D_values()[d]) > difference:
                Result_List.append(d)
        return Result_List


    S1 = All_Average_good().copy()
    S2 = price_good(price_dict = Dic_Close , day1 = 5 , day2 = 10 , day3 = 20 ,day4 = 60).copy()
    S3 = MACD_good(n = 9).copy()
    S4 = Volume_good(day = 15).copy()
    S5 = Gold_Cross(difference = 0.1).copy()


    result = list(set(S1).intersection(set(S2)))


    Date = [date for date in Dic_Close.keys()][26:]
    Prices = [float(price) for price in Dic_Close.values()][26:]

    R = []

    for d in range(1,len(Date)):
        if Prices[d-1] < Prices[d]:
            R.append(Date[d])

    def Poss(X,Y):
        X ,Y = set(X),set(Y)
        Result_correct = len(X.intersection(Y))/len(Y)
        Result_error = len(X.difference(Y))/len(X)
        return [Result_correct,Result_error]
    SS = [S1,S2,S3,S4,S5]  # SS == Union
    spereate_line = '='*70
    Result_Str += spereate_line
    Result_Str +='\n'
    for j in range(5):
        Union_Set_for_1 = set(SS[j])
        Result = Poss(Union_Set_for_1,R)
        Result_Str += ('S{} {}：{:.2%} {}：{:.2%}'.format(j+1,' 命中率',Result[0],' 失誤率',Result[1]).replace('S1','向上開花').replace('S2','突破   ')          .replace('S3','MACD  ').replace('S4','帶量   ').replace('S5','黃金交叉'))
        Result_Str += '\n'
    Result_Str += spereate_line
    Result_Str +='\n'

    for i in range(5):
        for j in range(5):
            if i < j :
                Union_Set_for_2 = set(SS[i]).union(set(SS[j]))
                Result = Poss(Union_Set_for_2,R)
                Result_Str +=('S{} S{} {}：{:.2%} {}：{:.2%}'.format(i+1,j+1,' 命中率',Result[0],' 失誤率',Result[1]).replace('S1','向上開花').replace('S2','突破   ')                  .replace('S3','MACD  ').replace('S4','帶量   ').replace('S5','黃金交叉'))
                Result_Str += '\n'
    Result_Str += spereate_line
    Result_Str +='\n'

    for i in range(5):
        for j in range(5):
            for k in range(5):
                if i < j < k :
                    Union_Set_for_3 = set(SS[i]).union(set(SS[j]).union(set(SS[k])))
                    Result = Poss(Union_Set_for_3,R)
                    Result_Str +=('S{} S{} S{} {}：{:.2%} {}：{:.2%}'.format(i+1,j+1,k+1,' 命中率',Result[0],' 失誤率',Result[1]).replace('S1','向上開花').replace('S2','突破   ')                      .replace('S3','MACD  ').replace('S4','帶量   ').replace('S5','黃金交叉'))
                    Result_Str += '\n'
    Result_Str += spereate_line
    Result_Str +='\n'
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for y in range(5):
                    if i < j < k < y :
                            Union_Set_for_4 = set(SS[i]).union(set(SS[j]).union(set(SS[k])).union(set(SS[y])))
                            Result = Poss(Union_Set_for_4,R)
                            Result_Str +=('S{} S{} S{} S{} {}：{:.2%} {}：{:.2%}'.format(i+1,j+1,k+1,y+1,' 命中率',Result[0],' 失誤率',Result[1]).replace('S1','向上開花').replace('S2','突破   ')                              .replace('S3','MACD  ').replace('S4','帶量   ').replace('S5','黃金交叉'))
                            Result_Str += '\n'
    Result_Str += spereate_line
    Result_Str +='\n'

    for i in range(5):
        for j in range(5):
            for k in range(5):
                for y in range(5):
                    for z in range(5):
                        if i < j < k < y <z:
                                Union_Set_for_5 = set(SS[i]).union(set(SS[j]).union(set(SS[k])).union(set(SS[y])).union(set(SS[z])))
                                Result = Poss(Union_Set_for_5,R)
                                Result_Str +=('S{} S{} S{} S{} S{} {}：{:.2%} {}：{:.2%}'.format(i+1,j+1,k+1,y+1,z+1,' 命中率',Result[0],' 失誤率',Result[1]).replace('S1','向上開花').replace('S2','突破   ')                                  .replace('S3','MACD  ').replace('S4','帶量   ').replace('S5','黃金交叉'))
                                Result_Str += '\n'
    Result_Str += spereate_line
    Result_Str +='\n'


    return print(Result_Str)

In [2]:
general(stockiid = '2330')

向上開花  命中率：35.07%  失誤率：51.92%
突破     命中率：55.91%  失誤率：29.72%
MACD    命中率：1.60%  失誤率：55.56%
帶量     命中率：40.08%  失誤率：52.83%
黃金交叉  命中率：30.86%  失誤率：31.25%
向上開花 突破     命中率：61.32%  失誤率：42.16%
向上開花 MACD    命中率：36.07%  失誤率：52.13%
向上開花 帶量     命中率：60.32%  失誤率：51.53%
向上開花 黃金交叉  命中率：58.72%  失誤率：45.84%
突破    MACD    命中率：56.71%  失誤率：30.64%
突破    帶量     命中率：69.74%  失誤率：46.21%
突破    黃金交叉  命中率：65.53%  失誤率：33.13%
MACD   帶量     命中率：41.08%  失誤率：52.98%
MACD   黃金交叉  命中率：31.86%  失誤率：33.47%
帶量    黃金交叉  命中率：56.51%  失誤率：49.55%
向上開花 突破    MACD    命中率：61.92%  失誤率：42.46%
向上開花 突破    帶量     命中率：73.75%  失誤率：48.53%
向上開花 突破    黃金交叉  命中率：70.94%  失誤率：43.00%
向上開花 MACD   帶量     命中率：60.92%  失誤率：51.75%
向上開花 MACD   黃金交叉  命中率：59.32%  失誤率：46.28%
向上開花 帶量    黃金交叉  命中率：73.75%  失誤率：49.93%
突破    MACD   帶量     命中率：70.34%  失誤率：46.33%
突破    MACD   黃金交叉  命中率：66.13%  失誤率：33.87%
突破    帶量    黃金交叉  命中率：77.56%  失誤率：46.40%
MACD   帶量    黃金交叉  命中率：57.11%  失誤率：49.91%
向上開花 突破    MACD   帶量     命中率：74.35%  失誤率：48.61%
向上開花 突破    MACD   黃金交叉  命中率：71.34%

In [44]:
import pandas as pd
def general(stockiid = '2330',MA = 5,MACD_int = 9,RSV_int = 9,Mean_Volume_int = 15):
    stockiid = int(stockiid)
    MA ,MACD_int,RSV_int,Mean_Volume_int= int(MA),int(MACD_int),int(RSV_int),int(Mean_Volume_int)
    CSV = './all_stock_price/public/{}.TW.csv'.format(stockiid)
    
    daily_trade =pd.read_csv('{}'.format(CSV))
    daily_trade = daily_trade.to_dict()

    Dic_Open = {}
    Dic_High = {}
    Dic_Low = {}
    Dic_Close = {}
    Dic_Volume = {}


    for count in range(len(daily_trade['Date'])):
        Dic_Open[daily_trade['Date'][count]] = daily_trade['Open'][count]
        Dic_High[daily_trade['Date'][count]] = daily_trade['High'][count]
        Dic_Low[daily_trade['Date'][count]] = daily_trade['Low'][count]
        Dic_Close[daily_trade['Date'][count]] = daily_trade['Close'][count]
        Dic_Volume[daily_trade['Date'][count]] = daily_trade['Volume'][count]

    L = [l for l in Dic_Close.values()]
    D = [d for d in daily_trade['Date'].values()]


    def Moving_Average(interval = 5):
        Dic_MA = {}
        interval  =int(interval)
        for i in range(len(L)):
            if i-interval <0:
                pass
            else:
                Dic_MA[D[i]]="{:}".format(sum(L[i-interval:i])/interval)
        return Dic_MA


    def Bias(days = 3,date = '2020-02-19'):
        try:
            x = Dic_Close[date]
            y = float( Moving_Average( interval = days )[date] )
            Result = (x-y)/y
        except KeyError as f:
            Result = 0
        return Result


    def EMA(day = 12):

        Result_List = []
        Result_Dic = {}
        day  =int(day)
        for i in range(len(L)):
            denominator_EMA = 0
            fraction_EMA = 0
            if i-day < 0:
                Result_List.append('NaN')
            else:
                interval_price = L[i - day:i]
                alpha = ((2/(day+1)))
                for c in range(day):
                    denominator_EMA += (1-alpha)**c
                    fraction_EMA += ((1-alpha)**c)*float(interval_price[c])
                Result_List.append (fraction_EMA / denominator_EMA )
                Result_Dic = dict(zip(D,Result_List))
        return Result_Dic



    def DIF(day1 = 12,day2 = 26):
        day1 = int(day1)
        day2 = int(day2)
        Result_List = []
        EMA_1 = [e for e in EMA(day = day1).values()]
        EMA_2 = [e for e in EMA(day = day2).values()]
        for c in range(len(EMA_1)):
            try:
                Result_List.append(EMA_1[c] - EMA_2[c])
            except TypeError as e:
                Result_List.append('NaN')
            Result_Dic = dict(zip(D,Result_List))
        return Result_Dic

    def MACD(day = 9):
        mL = [l for l in DIF(day1 = 12,day2 = 26).values()]
        mResult_List = []
        mResult_Dic = {}
        day  =int(day)
        for i in range(len(L)):
            denominator_EMA = 0
            fraction_EMA = 0
            if i-day < 0:
                mResult_List.append('NaN')

            else:
                interval_price = mL[i - day:i]
                alpha = ((2/(day+1)))
                for c in range(day):
                    denominator_EMA += (1-alpha)**c
                    fraction_EMA += ((1-alpha)**c)*float(interval_price[c])
                mResult_List.append (fraction_EMA / denominator_EMA )
                mResult_Dic = dict(zip(D,mResult_List))
        return mResult_Dic


    def RSV(n = 9):
        Close_List = [l for l in Dic_Close.values()]
        High_List = [h for h in Dic_High.values()]
        Low_list = [low for low in Dic_Low.values()]
        D = [d for d in Dic_Close.keys()]
        day = int(n)
        Result_List = []
        for i in range(len(Close_List)):
            denominator_RSV = 0
            fraction_RSV = 0
            min_price = 0.0
            Max_price = 0.0
            Result = 0.0
            if i-day < 0:
                Result_List.append('NaN')
            else:
                interval_close_price = Close_List[i - day:i]
                interval_high_price = High_List[i - day:i]
                interval_low_price = Low_list[i - day:i]
                min_price = min(interval_low_price)
                Max_price = max(interval_high_price)

                denominator_RSV = Max_price - min_price
                fraction_RSV = Close_List[i] - min_price
                Result = fraction_RSV / denominator_RSV
                Result_List.append(Result)
        Result_dict = dict(zip(D,Result_List))
        return Result_dict


    def K_values(dic = RSV()):
        Days = [d for d in dic.keys()][9:]
        Null_list = ['NaN' for i in range(9)]
        K_value = 0.0
        K_list = []
        for day in Days:
            K_value = (K_value)*(2/3)+ (dic[day])*(1/3)
            K_list.append(K_value)
        K_list = ['NaN' for i in range(9)] + K_list
        Result_Dic = dict(zip([d for d in dic.keys()],K_list))
        return Result_Dic



    def D_values():
        Result_Dic = K_values(dic = K_values())
        return Result_Dic


    def Mean_Volume(interval = 15):
        L = [l for l in Dic_Volume.values()]
        D = [d for d in Dic_Volume.keys()]
        interval = int(interval)
        Dic_MV = {}
        for i in range(len(L)):
            if i - interval <0:
                pass
            else:
                Dic_MV[D[i]]="{}".format(sum(L[i-interval:i])/interval)
        return Dic_MV
    
    tmp1 = pd.Series( MACD(day = MACD_int),name='MACD_{}'.format(MACD_int)).to_frame()
    tmp2 = pd.Series( RSV(n = RSV_int),name='RSV_{}'.format(RSV_int)).to_frame()
    tmp3 = pd.Series( K_values(),name='K_values').to_frame()
    tmp4 = pd.Series( D_values(),name='D_values').to_frame()
    tmp5 = pd.Series( Mean_Volume(interval = Mean_Volume_int),name='Mean_Volume_{}'.format(Mean_Volume_int)).to_frame()
    
    
    
    Result_df = pd.Series( Moving_Average(interval = MA),name='MA_{}'.format(MA)).to_frame()
    Result_df['MACD_{}'.format(MACD_int)] = tmp1['MACD_{}'.format(MACD_int)]
    Result_df['RSV_{}'.format(RSV_int)] = tmp2['RSV_{}'.format(RSV_int)]
    Result_df['K_values'] = tmp3['K_values']
    Result_df['D_values'] = tmp4['D_values']
    Result_df['Mean_Volume_{}'.format(Mean_Volume_int)] = tmp5['Mean_Volume_{}'.format(Mean_Volume_int)]
    

    
    
    
    
    
    
    
    
    
    
    return Result_df


In [45]:
general(stockiid = '2330',MA = 5,)

,MA_5,MACD_9,RSV_9,K_values,D_values,Mean_Volume_15
2016-01-11,115.402,NaN,NaN,NaN,NaN,NaN
2016-01-12,114.3,NaN,NaN,NaN,NaN,NaN
2016-01-13,113.452,NaN,NaN,NaN,NaN,NaN
2016-01-14,113.11200000000001,NaN,NaN,NaN,NaN,NaN
2016-01-15,112.85600000000002,NaN,0.5,0.166667,0.0555556,NaN
2016-01-18,113.36599999999999,NaN,0.684015,0.339116,0.150076,NaN
2016-01-19,114.04400000000001,NaN,1,0.559411,0.286521,NaN
2016-01-20,114.89200000000001,NaN,0.533752,0.550858,0.374633,NaN
2016-01-21,115.06199999999998,NaN,0.599686,0.567134,0.4388,NaN
2016-01-22,115.65599999999999,NaN,1.06593,0.733401,0.537,NaN
